In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" "trl<0.9.0" "peft<0.12.0" \
                     "accelerate<0.32.0" "bitsandbytes<0.44.0" \
                     "transformers<4.43.0" "datasets" "tqdm" "pandas" "scikit-learn"


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-oiqhhlbb/unsloth_1a549e7521b247e9915d6e670c20ff3b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-oiqhhlbb/unsloth_1a549e7521b247e9915d6e670c20ff3b
  Resolved https://github.com/unslothai/unsloth.git to commit d707bd43b4e883b521761d525be2fae428fe5980
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 160.6 MB/s eta 0:00:00

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(20000))      # Use the first 5,000 for training
validation_dataset = shuffled_dataset.select(range(20000, 24000)) # Use the next 500 for validation

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# The instructional prompt template for training
training_prompt = """You are a math expert. Decide whether the proposed answer is correct.

Question:
{}

Proposed Answer:
{}

Explanation / Solution:
{}

Respond only with True if the proposed answer is correct, otherwise respond with False.

Output:
{}"""


EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, answer, solution, output in zip(questions, answers, solutions, outputs):
        text = training_prompt.format(question, str(answer), str(solution), str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # A small rank for lighter training
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64, # A common practice is to set alpha = 2 * r
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.12 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 20,
        max_steps = 2500,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1 | Total steps = 2,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.403500
20,0.876000
30,0.754800
40,0.740300
50,0.772000
60,0.753800
70,0.744200
80,0.808700
90,0.794400
100,0.775100


TrainOutput(global_step=2500, training_loss=0.6687122446060181, metrics={'train_runtime': 5889.9629, 'train_samples_per_second': 3.396, 'train_steps_per_second': 0.424, 'total_flos': 3.186781441184563e+17, 'train_loss': 0.6687122446060181, 'epoch': 1.0})

In [ ]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a math expert. Decide whether the proposed answer is correct.

Question:
{}

Proposed Answer:
{}

Explanation / Solution:
{}

Respond only with True if the proposed answer is correct, otherwise respond with False.

Output:
"""

# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = example["question"]
answer = example["answer"]
solution = example["solution"]

# Format the prompt with the validation data
inputs = tokenizer(
[
    inference_prompt.format(question, str(answer), str(solution))
], return_tensors = "pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### ANSWER ####")
print(answer)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

#### QUESTION ####
How many unordered pairs of prime numbers have a sum of 40?

#### ANSWER ####
18

#### SOLUTION ####
To compute all the prime numbers with sum of 40, we can enumerate all pairs of primes where sum = 40. 
One such instance can be solved by enumeration as follows:
<llm-code>
import itertools

answer = []
# Max sum of two primes below 40 is 39, so limiting ourselves to that
# Prevents us from examining the same pairing (i.e., [3,39]) again
for p1, p2 in itertools.product(range(2,20), range(2,39)):
    if p1 + p2 == 40:
        answer.append((p1, p2))

print(len(answer))
</llm-code>
<llm-code-output>
18
</llm-code-output>
Hence, there are \boxed{18} unordered pairs of prime numbers that have a sum of 40.

#### MODEL'S PREDICTION ####
False<|end_of_text|>

#### CORRECT ANSWER ####
False


In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    answer = example["answer"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(answer), str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")


    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

 13%|█▎        | 1274/10000 [05:43<39:05,  3.72it/s]Unsloth: Input IDs of shape torch.Size([1, 1186]) with length 1186 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 16%|█▌        | 1572/10000 [07:03<37:04,  3.79it/s]Unsloth: Input IDs of shape torch.Size([1, 1028]) with length 1028 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 23%|██▎       | 2329/10000 [10:24<33:48,  3.78it/s]Unsloth: Input IDs of shape torch.Size([1, 1201]) with length 1201 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 30%|███       | 3050/10000 [13:36<30:43,  3.77it/s]Unsloth: Input IDs of shape torch.Size([1, 1245]) with length 1245 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
 33%|███▎      | 3251/10000 


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


In [ ]:
!zip -r /content/checkpoint-60.zip /content/outputs/checkpoint-60


  adding: content/outputs/checkpoint-60/ (stored 0%)
  adding: content/outputs/checkpoint-60/adapter_config.json (deflated 57%)
  adding: content/outputs/checkpoint-60/scheduler.pt (deflated 62%)
  adding: content/outputs/checkpoint-60/adapter_model.safetensors (deflated 8%)
  adding: content/outputs/checkpoint-60/special_tokens_map.json (deflated 71%)
  adding: content/outputs/checkpoint-60/training_args.bin (deflated 53%)
  adding: content/outputs/checkpoint-60/rng_state.pth (deflated 26%)
  adding: content/outputs/checkpoint-60/trainer_state.json (deflated 66%)
  adding: content/outputs/checkpoint-60/scaler.pt (deflated 64%)
  adding: content/outputs/checkpoint-60/optimizer.pt (deflated 11%)
  adding: content/outputs/checkpoint-60/tokenizer.json (deflated 85%)
  adding: content/outputs/checkpoint-60/README.md (deflated 65%)
  adding: content/outputs/checkpoint-60/tokenizer_config.json (deflated 96%)


In [ ]:
!rm -rf /content/outputs/checkpoint-*
